<a href="https://colab.research.google.com/github/HenryLau08/adsai-mlops-2025/blob/main/MLOPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %pip install kaggle

In [ ]:
# %pip install ultralytics

In [ ]:
# %pip install mlflow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# imports
import os
from pathlib import Path
import cv2
import shutil
import zipfile
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import numpy as np
import seaborn as sns
from PIL import Image
from concurrent.futures import ThreadPoolExecutor

from sklearn.model_selection import train_test_split

data_dir = Path("/content/drive/MyDrive/Colab Notebooks/data")
data_dir.mkdir(parents=True, exist_ok=True)
os.makedirs("/root/.config/kaggle/", exist_ok=True)


In [ ]:
# run it once
shutil.move("kaggle.json", "/root/.config/kaggle/") # manually upload your kaggle.json file and run it once
!chmod 600 /root/.config/kaggle/kaggle.json
import kaggle

Groepsnaam

Groepsleden

michal
quincy
henry
moh
jayden

Opdrachtgever

Tesla


# 1. INLEIDING

opdrachtbeschrijving


Beschrijf het doel van de datapijplijn en geef een korte samenvatting van de stappen die je gaat volgen.


# 2. Data Ingestion
Beschrijf hoe de data wordt ingeladen vanuit verschillende bronnen (Kaggle, Google Images, live camera feeds).


Kaggle Datasets

Beschrijf hoe je de datasets van Kaggle downloadt en inlaadt.

Gebruik bijvoorbeeld de Kaggle API om de datasets te downloaden.

Google Images

Beschrijf hoe je afbeeldingen van Google Images haalt via web scraping of een API.

Leg uit hoe je de afbeeldingen opslaat en organiseert.

Live Camera Feeds

Beschrijf hoe je live videofeeds verwerkt voor realtime voertuigdetectie.

Leg uit hoe je frames uit de videofeeds haalt en deze gebruikt als input voor het model.



## 2.1 Kaggle Datasets

In [ ]:
kaggle_path = str(data_dir + '/kaggle')
os.makedirs(kaggle_path, exist_ok=True)

kaggle.api.dataset_download_files("sakshamjn/vehicle-detection-8-classes-object-detection", path=kaggle_path, unzip=True)

Dataset URL: https://www.kaggle.com/datasets/sakshamjn/vehicle-detection-8-classes-object-detection


In [8]:
images_path = "/content/drive/MyDrive/Colab Notebooks/data/kaggle/train/images"
labels_path = "/content/drive/MyDrive/Colab Notebooks/data/kaggle/train/labels"

images = [f for f in os.listdir(images_path) if f.endswith('.jpg')]
labels = [f.replace('.jpg', '.txt') for f in images]

In [ ]:
def extract_labels_and_bboxes(label_file):
    with open(label_file, 'r') as f:
        for line in f:
            parts = line.strip().split(' ')
            yield int(parts[0]), [float(x) for x in parts[1:]]

In [ ]:
def load_kaggle_dataset(images_path, labels_path):
    data = []
    images = [f for f in os.listdir(images_path) if f.endswith('.jpg')]
    labels = [f.replace('.jpg', '.txt') for f in images]

    for img, lbl in tqdm(zip(images, labels), total=len(images)):
        label_file = os.path.join(labels_path, lbl)
        for label, bbox in extract_labels_and_bboxes(label_file):
            data.append({
                'image_path': os.path.join(images_path, img),
                'label_path': label_file,
                'label': label,
                'x_center': bbox[0],
                'y_center': bbox[1],
                'width': bbox[2],
                'height': bbox[3]
            })
    return pd.DataFrame(data)

In [ ]:
csv_path = os.path.join(data_dir, 'kaggle_data.csv')
if not os.path.exists(csv_path):
    df_kaggle = load_kaggle_dataset(images_path, labels_path)
    df_kaggle.to_csv(csv_path, index=False)
else:
    df_kaggle = pd.read_csv(csv_path)

In [ ]:
display(df_kaggle.head(10))

In [ ]:
def get_image_size(path):
    try:
        with Image.open(path) as img:
            return img.size
    except Exception as e:
        print(f"Fout bij openen van {path}: {e}")
        return None

with ThreadPoolExecutor(max_workers=8) as executor:
    sizes = list(executor.map(get_image_size, df_kaggle['image_path']))

# Remove None values and get unique sizes
image_sizes = sorted(set(s for s in sizes if s is not None))

print(image_sizes)

In [ ]:
def show_image_with_all_bboxes(df, class_names=None, filter_label=None, amount=5, shuffle=False):

    if filter_label is not None:
        unique_images = df[df['label'] == filter_label]['image_path'].unique()
    else:
        unique_images = df['image_path'].unique()

    if shuffle:
        np.random.shuffle(unique_images)

    for image_path in unique_images[:amount]:
        # Filter rows for the specific image
        image_rows = df[df['image_path'] == image_path]

        # Load the image
        image = cv2.imread(image_path)
        if image is None:
            print(f"Image not found: {image_path}")
            return
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w, _ = image.shape

        # Draw all bounding boxes for this image
        for _, row in image_rows.iterrows():
            cls = int(row['label'])
            x_center, y_center = row['x_center'], row['y_center']
            width, height = row['width'], row['height']

            x1 = int((x_center - width / 2) * w)
            y1 = int((y_center - height / 2) * h)
            x2 = int((x_center + width / 2) * w)
            y2 = int((y_center + height / 2) * h)

            # Use different color for filter_label if provided
            if filter_label is not None and cls == filter_label:
                color = (0, 255, 0)  # Green for filtered label
            else:
                color = (255, 0, 0)  # Red for others

            cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)

            label_text = class_names[cls] if class_names else str(cls)
            cv2.putText(image, label_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        # Show the result
        plt.figure(figsize=(10, 10))
        plt.imshow(image)
        plt.axis('off')
        plt.show()

In [ ]:
show_image_with_all_bboxes(df_kaggle, filter_label=0)

Kaggle dataframe Labels:  
0 = Auto  
1 = Bus  
2 = Car  
3 = LCV  
4 = Motorcycle  
5 = Multi-Axle  
6 = Tractor  
7 = Truck

## 2.2 Google Images


## 2.3 Live Camera Feeds


# 3. Data Opschonen en Transformeren

Beschrijf de stappen voor data cleaning, data augmentation en data transformation.

In deze sectie beschrijf je de stappen voor het opschonen en transformeren van de data. Gebruik markdown-cellen om elke stap te beschrijven en code-cellen om de transformaties uit te voeren.

Data Cleaning

Beschrijf hoe je de afbeeldingen opschoont (bijvoorbeeld door ze allemaal naar hetzelfde formaat te converteren).

Voer kleurnormalisatie uit en pas cropping en padding toe waar nodig.

Data Augmentation

Beschrijf hoe je data augmentation toepast om de dataset te vergroten en het model robuuster te maken.

Gebruik technieken zoals rotatie, spiegeling, schalen, translatie, shearing en cropping.

Data Transformation

Beschrijf hoe je de afbeeldingen transformeert om ze geschikt te maken voor machine learning (bijvoorbeeld normalisatie en kleurtransformaties).

## 3.1 Data Cleaning

## 3.2 Data Augmentation

## 3.3 Data Transformation

# 4. Machine Learning Pipeline

Beschrijf hoe het objectdetectiemodel wordt getraind en geëvalueerd.


In deze sectie beschrijf je hoe je het objectdetectiemodel traint en evalueert. Gebruik markdown-cellen om elke stap te beschrijven en code-cellen om het model te trainen en te evalueren.

Objectdetectie Model

Beschrijf hoe je een objectdetectiemodel (bijvoorbeeld YOLO, Faster R-CNN of EfficientDet) traint op de dataset.

Leg uit hoe je het model evalueert met behulp van metrieken zoals precisie, recall en F1-score.

Image Captioning

Beschrijf eventueel hoe je een image captioning-model toevoegt om beschrijvingen van gedetecteerde voertuigen te genereren.



## 4.1 Train-Test Split

## 4.2 Model Bouwen

## 4.3 Model Trainen

## 4.4 Model Evalueren

# 5. Deployment

Beschrijf hoe het model wordt gedeployed op edge-apparaten en in de cloud.

In deze sectie beschrijf je hoe je het model deployt op edge-apparaten en in de cloud. Gebruik markdown-cellen om de deploymentstrategie te beschrijven.

Edge Deployment

Beschrijf hoe je het model implementeert op edge-apparaten zoals slimme camera's of embedded systemen.

Leg uit hoe je ervoor zorgt dat het model realtime voertuigdetectie kan uitvoeren.

Cloud Deployment

Beschrijf hoe je het geavanceerde deep learning-model in de cloud deployt voor voertuigclassificatie.



## 5.1 Edge Deployment

## 5.2 Cloud Deployment

# 6. Monitoring en Feedback

Beschrijf hoe de prestaties van het model worden gemonitord en hoe het model wordt hertraind op basis van nieuwe data.

In deze sectie beschrijf je hoe je de prestaties van het model monitort en hoe je het model hertraind op basis van nieuwe data. Gebruik markdown-cellen om de monitoringstrategie te beschrijven.

Data Drift Detection

Beschrijf hoe je veranderingen in de inkomende data detecteert en hoe je hierop reageert.

Model Performance Monitoring

Beschrijf hoe je de nauwkeurigheid, recall en precision van het model continu monitort.

Leg uit hoe je automatisch logging van verkeerde voorspellingen implementeert.

Hertraining

Beschrijf hoe je het model periodiek hertraind op basis van nieuwe data en feedback.

7. Conclusie


## 6.1 Data Drift Detection

## 6.2 Hertraining

# 7. Conclusie

Geef een samenvatting van de datapijplijn en bespreek eventuele uitdagingen of verbeterpunten.

In deze sectie geef je een samenvatting van de datapijplijn en bespreek je eventuele uitdagingen of verbeterpunten. Gebruik markdown-cellen om de conclusie te schrijven.




# 8. Bronvermelding